In [21]:
import matplotlib.pyplot as plt
import numpy as np
import math as mt
import seaborn as sns
from tqdm import tqdm
import pandas as pd
from hangul_utils import split_syllables, join_jamos
from tqdm.auto import tqdm
import re
#from eunjeon import Mecab
from konlpy.tag import Mecab
from khaiii import KhaiiiApi
from hanspell import spell_checker

In [22]:
con_dict = [
    
    ['ㅏㅣ','ㅐ'], ['ㅑㅣ','ㅒ'], ['ㅓㅣ','ㅔ'],
    ['ㅕㅣ','ㅖ'], ['ㅗㅣ','ㅚ'], ['ㅗㅐ','ㅙ'],
    ['ㅜㅓ','ㅝ'], ['ㅜㅔ','ㅞ'], ['ㅡㅣ','ㅢ'],
    ['ㅣㅏ','ㅑ'], ['ㅣㅓ','ㅕ'], ['ㅣㅗ','ㅛ'],
    ['ㅣㅜ','ㅠ'], ['ㅡㅓ','ㅓ'], ['ㅗㅏ','ㅘ']
    
]

jongsung_list = [ 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']


#lis_beta = ['EP+EF', 'VCP+EF', 'B+EF', 'B+EP+EF', 'B+VCP+EF', 'EF','EP']

lis_beta = ['EP+EF', 'EF', 'B+EF', 'B+EP+EF']

#어말을 처리해 주기 위한 것으로, 나중에 EC등이 필요해 진다면 이 부분에 EC 등을 집어넣어준다. 참고로 말하자면 이는 마지막에 위치해야한다.
#특이 바로 밑의 이 부분은 형태소 태그가 이 리스트 안의 것과 일치하는 경우 단순 삭제를 하는 것이고
lis_beta_ef = ['EP+EP+EF', 'EP+EF', 'EF', 'UNKNOWN']

lis_beta_ef_h = ['EF', 'UNKNOWN']

#이 부분 같은 경우는 마지막에 오는 것을 처리하는 것으로 단독으로만 들어가는게 좋겠지?
lis_tag_last = ['EF', 'UNKNOWN']

lis_end = [
    
    'ㅅㅡㅂㄴㅣㄷㅏ',
    'ㅂㄴㅣㄷㅏ', 'ㄴㅣㄷㅏ', 'ㅂㅅㅣㄷㅏ', 'ㅅㅣㄷㅏ', 'ㄹㄹㅐㅇㅛ',
    'ㅅㅔㅇㅛ', 'ㄷㅔㅇㅛ', 'ㅇㅔㅇㅛ', 'ㅇㅖㅇㅛ', 'ㄴㅏㅇㅛ', 'ㅇㅡㄹㄲㅏㅇㅛ', 'ㅇㅣㄹㄲㅏㅇㅛ', 'ㄹㄲㅏㅇㅛ', 'ㅇㅡㄴㄱㅏㅇㅛ', 'ㅇㅣㄴㄱㅏㅇㅛ','ㅇㅛ',
    'ㅈㅛ',
    'ㅅㅣㅂㅅㅣㅇㅗ', 'ㅅㅣㅇㅗ', 'ㅇㅗ',
    'ㅂㄴㅣㄲㅏ',  
    
]

lis_end_2low = [
    
    'ㄷㅓㄹㅏ','ㄴㄷㅏ', 'ㅆㄷㅏ', 'ㄹㅗㄷㅏ', 'ㄷㅏ', 'ㄱㅔ', 'ㅈㅣ', 'ㅈㅏ', 
    'ㄹㅏ',
    'ㅇㅑ', 
    'ㄴㅣㄲㅏ', 
    'ㄲㅏ', 
    'ㄴㅣ', 
    'ㅇㅏ', 
    'ㄷㅔ', 
    
]

lis_wk = [
    
    ['ㄱㅖ', 'ㅇㅣㅆㅇㅡ'], ['ㅈㅜㅁㅜ','ㅈㅏ'], ['ㅈㅏㅂㅅㅜ','ㅁㅓㄱㅇㅡ']
    
]

# lis_end_2low = [
    
#     'ㄷㅓㄹㅏ','ㄴㄷㅏ', 'ㅆㄷㅏ', 'ㄷㅏ', 
#     'ㄹㅏ',
#     'ㅇㅑ', 
    
# ]

P_LIST = ['.', '?', '!', '\'', '\"', 'ᆞ', 'ᆢ', 'ㆍ',  '”', '’',')', '(', ',', '”']

SV_LIST = ['\'', '\"', ':', ';']

lis_plus = [
    
    'EP', 'VCP', 
    
]

In [23]:
mec = Mecab()
khai = KhaiiiApi()

In [24]:
def detect_h(input, lis_end_h, lis_end_l):
    for i in lis_end_h:
        if len(input)>=len(i):
            if input[-len(i):]==i:
                return 1
            
    for i in lis_end_l:
        if len(input)>=len(i):
            if input[-len(i):]==i:
                return 0
            
    return -1

def unite(input, dict):
    for i in dict:
        input = re.sub(i[0],i[1],input)
    return input
    
## 자모 단위로 문장을 나누고 합칠 때 쓰는 class ##
class Jamodealer:
    jamo = []
    pp = ''
    #각 단어들을 받아와서 자모단위로 나눈다.
    def __init__(self,lis_word):
    
        self.jamo = []
        for i in lis_word:
            self.jamo.append(split_syllables(i))
    
    ##사전에서 변환된 자모단위로 분리된 문장을 합칠 때 쓰는 함수이다.     
    def make_one(self):
        #list 형태로 저장된 자모들의 집합을 하나의 string pp에 저장한다. 
        self.pp = ''
        for i in self.jamo:
             self.pp= self.pp+i
        ##종성과 종성을 합쳐야 하는 경우가 있다면 합친다.        
        self.pp = unite(self.pp, con_dict)
        
        #자모 단위의 string에서 자모 단위로 사전을 만들고 거기에 index를 부여한다.        
        chars = list(set(self.pp))
        char_to_ix = { ch:i for i,ch in enumerate(chars) }
        ix_to_char = { i:ch for i,ch in enumerate(chars) }
        
        #자모 단위로 분리되었던 문장을 다시 하나로 합친다.
        jamo_numbers = [char_to_ix[x] for x in self.pp]
        restored_jamo = ''.join([ix_to_char[x] for x in jamo_numbers])
        #합쳐진 문장을 return 한다.
        restored_text = join_jamos(restored_jamo)
        return restored_text

In [25]:
def prepro_ch01(input, lis_ef, tag_last, lis_w_last, lis_w_last_not):
    
    lis_res_word = []
    
    lis_input = input.split()
    lis_word = []
    lis_tag = []
    lis_last_word = []
    lis_ind = []
    
    lis_result = []
    
    for i in range(len(lis_input)):
        ele_w = []
        ele_t = []
        
        an = mec.pos(lis_input[i])
        for j in range(len(an)):
            ele_w.append(an[j][0])
            ele_t.append(an[j][1])
        elem_w = ' '.join(ele_w)
        elem_t = '/'.join(ele_t)
        
        jam_pre = Jamodealer(elem_w)
        lis_word.append(''.join(jam_pre.jamo))
        lis_tag.append(elem_t)
    
    for i in range(len(lis_tag)):
        
        if 'EF/SF' in lis_tag[i] or 'EF/SV' in lis_tag[i]:
            elemen_t = lis_tag[i].split('/')
            elemen_w = lis_word[i].split(' ')
            flag = 0
            
            
            for j in range(len(elemen_t)):

                flag_end = detect_h(elemen_w[j], lis_w_last_not, lis_w_last)

                
                if flag_end == 0 and 'EF' in elemen_t[j]:
                    
                    for jam in lis_w_last:
                        
                        if len(elemen_w[j])>=len(jam):
                            
                            if elemen_w[j][-len(jam):]==jam:
                                
                                
                                lis_ind.append(i)
                                lis_last_word.append(jam)
                                elemen_w[j] = elemen_w[j][:-len(jam)]
                                
                                break
                                
            res_w = ''.join(elemen_w)
            lis_result.append(res_w)
                                
        else:
            
            rere = lis_word[i].split(' ')
            
            resres = ''.join(rere)
            
            lis_result.append(resres)
            
            
    return lis_result, lis_tag, lis_ind, lis_last_word
                

In [26]:
print(prepro_ch01('이번주 금요일에 그곳에 갈 것이다.', lis_beta_ef, lis_tag_last, lis_end_2low, lis_end))

(['ㅇㅣㅂㅓㄴㅈㅜ', 'ㄱㅡㅁㅇㅛㅇㅣㄹㅇㅔ', 'ㄱㅡㄱㅗㅅㅇㅔ', 'ㄱㅏㄹ', 'ㄱㅓㅅㅇㅣ.'], ['NNG/NNG', 'NNG/JKB', 'NP/JKB', 'VV+ETM', 'NNB/VCP/EF/SF'], [4], ['ㄷㅏ'])


In [27]:
def treatSF(stc, ex):
    ind_point = -1
    point = ''          
    for i in range(len(stc)):
        if stc[i] in P_LIST:
            point = stc[i]
            break
            
    if point in P_LIST:
        ind_point = stc.index(point)
    
    r_word = ''
    r_pun = ''
    
    if ind_point!=-1:
        r_word = stc[:ind_point]
        r_pun = stc[ind_point:]
    else:
        r_word = stc
    return r_word+ex+r_pun

def convert_not(stc, tag, ex):
    return treatSF(stc, ex)

def convert_an(stc, tag, ex):
    
    ind_point = -1
    point = ''
            
    for i in range(len(stc)):
        if stc[len(stc) - i] in P_LIST:
            point = stc[len(stc) - i]
            break
            
    if point in P_LIST:
        ind_point = stc.index(point)
    
    r_word = ''
    r_pun = ''
    
    if ind_point!=-1:
        r_word = stc[:ind_point]
        r_pun = stc[ind_point:]
    else:
        r_word = stc
    
    if ex=='ㄹㅏ':
        
        if len(r_word)!=0:
            if 'ㅗ'==stc[-2]:
                return r_word+'ㅏㅇㅛ'+r_pun
            elif 'ㅎㅏ'==stc[-3:-1]:
                return r_word+'ㅣㅇㅛ'+r_pun
            elif 'ㄷㅏㄹ' in stc:
                return r_word+'ㄹㅏ'+r_pun
                if 'VA' not in tag:
                    return r_word+'ㄹㅏㅇㅛ'+r_pun
                return r_word[:-3]+'ㅈㅜㅅㅔㅇㅛ'+r_pun
            elif 'ㅗㄹ' == r_word[-2:]:
                return r_word+'ㄹㅏㅇㅛ'+r_pun
            else:

                return r_word+'ㅇㅛ'+r_pun
        else:
            return r_word+ex+r_pun

        #return r_word+'ㅇㅛ'+r_pun
    elif ex=='ㄱㅜㄴㅏ':
        return r_word+'ㄱㅜㄴㅇㅛ'+r_pun
    elif ex=='ㅈㅏㄶㄴㅣ':
        return r_word+'ㅈㅏㄶㅇㅏㅇㅛ'+r_pun
    elif ex=='ㄱㅔ':###############################
        print(r_word)
        if 2<=len(r_word) and r_word[-2:]=='ㅎㅏ':
            return r_word+'ㅣㅇㅛ'+r_pun
        elif 2<=len(r_word) and r_word[-2:]=='ㅅㅣ':
            return r_word+'ㅓㅇㅛ'+r_pun
        return r_word+'ㄱㅔㅇㅛ'+r_pun
        
    elif ex=='ㅇㅓㄹㅏ':
        return r_word+'ㅇㅓㅇㅛ'+r_pun
    elif ex=='ㅇㅏㄹㅏ':
        return r_word+'ㅇㅏㅇㅛ'+r_pun
    elif ex=='ㄴㅗㄹㅏ':
        if stc[-3:-1]=='ㅎㅏ':
            return r_word+'ㅣㅇㅛ'+r_pun
        else:
            return r_word+'ㅇㅓㅇㅛ'+r_pun
    elif ex=='ㄷㅏㄷㅓㄹㅏ':
        return r_word+'ㄷㅐㅇㅛ'+r_pun
    elif ex=='ㄷㅓㄹㅏ':
        wd = ''
        if 'ㄷㅏ' in r_word:
#             print('dd')
            return r_word[:-1]+'ㅐㅇㅛ'+r_pun
        if r_word[-1] in jongsung_list:
            wd = r_word[-2]
        else:
            wd = r_word[-1]
        if wd=='ㅗ':
            return r_word+'ㅏㅇㅛ'+r_pun
        elif wd=='ㅜ':
            return r_word+'ㅓㅇㅛ'+r_pun
        elif wd=='ㅏ':
            return r_word+'ㅇㅏㅇㅛ'+r_pun
        else:
            return r_word+'ㅇㅓㅇㅛ'+r_pun
    elif ex=='ㄴㅣ' or ex =='ㄴㅡㄴㄱㅏ' or ex =='ㅇㅡㄴㄱㅏ':
        
        isVcp = tag[-11:-3]
        
        if isVcp =='VCP' or isVcp =='/VCP' or isVcp =='NNG/VCP':
            return r_word + 'ㅇㅖㅇㅛ' + r_pun
        elif isVcp == 'VCP/NULL':
            return r_word + 'ㅇㅔㅇㅛ' + r_pun
        
        
        if ex=='ㄴㅣ' and stc[-3:-1]=='ㅌㅔ':
            return r_word+'ㄴㅣㅇㅛ'+r_pun
        
        if ex=='ㄴㅣ' and 'NNG' in tag:
            return r_word+'ㅇㅖㅇㅛ'+r_pun
        
        return r_word+'ㄴㅏㅇㅛ'+r_pun
    elif ex=='ㅇㅑ':
        return r_word+'ㅇㅔㅇㅛ'+r_pun
#     elif ex=='ㄷㅓㄹㅏ':
#         if 'ㄷㅏ' in stc:
#             return stc[:-1]+'ㅔㅇㅛ'
#         return stc+'ㄷㅔㅇㅛ'
    elif ex=='ㅈㅣ':
        return r_word+'ㅈㅛ'+r_pun
    elif ex=='ㅈㅏ':
        return r_word+'ㅈㅛ'+r_pun
    else:
        return r_word+ex+'ㅇㅛ'+r_pun
    
def convert_da(stc, tag, ex):
    #print(tag)
    if 'UNKNOWN/' in tag:
        isVcp = tag[-17:]
    else:
        isVcp = tag[-9:]

    isETM = tag[-12:]


    ind_point = -1
    point = ''

    for i in range(len(stc)):
        if len(stc)-i-1==0:
            point = stc[0]
            ind_pont = -1
        if stc[len(stc) - i-1] not in P_LIST:
            point = stc[len(stc) - i-1]
            ind_point = len(stc)-i

            break
    
#     print(ind_point)
    
#     if point in P_LIST:
#         ind_point = stc.index(point)
    
    r_word = ''
    r_pun = ''

    if ind_point!=-1:
        r_word = stc[:ind_point]
        r_pun = stc[ind_point:]
    else:
        r_word = ''
        r_pun = stc[0]
    
    try:
        jongsung = r_word[-1]
    
    except:
        return ex+r_pun
        
    if jongsung in jongsung_list:
        if  isETM.find('+ETM') != -1:
            r_word = r_word[:-1]
            final = r_word +'ㅂㄴㅣㄷㅏ'+r_pun
        else:
            final = r_word +'ㅅㅡㅂㄴㅣㄷㅏ'+r_pun
    else:
        if isVcp.find('VCP') != -1:
            if len(r_word)>=2:
                if 'ㅇㅣ' in r_word:
                    return r_word +'ㅂㄴㅣㄷㅏ'+r_pun
            final = r_word +'ㅇㅣㅂㄴㅣㄷㅏ'+r_pun
        elif 'SN/NULL' in tag:
            final = r_word +'ㅇㅣㅂㄴㅣㄷㅏ'+r_pun
        else:
            final = r_word +'ㅂㄴㅣㄷㅏ'+r_pun
    #print(final)
    return final

In [28]:
class Changer(object):
    
    def make_end_h(self, stc, tag, ex):

#         print(ex)
        if 'ㄷㅏ'== ex[-2:]:

            return convert_da(stc, tag, ex)
            #return convert_not(stc, tag, ex)
        else:
            return convert_not(stc, tag, ex)
            #return convert_an(stc, tag, ex)

    
    def to_high(self, input):
        
        if '  ' in input:
            return input
        
        result = input
        
        

        
        lis_w, lis_t, lis_target_ind, off_word = prepro_ch01(result, lis_beta_ef, lis_tag_last, lis_end_2low, lis_end)

        
        if len(lis_target_ind)!=0:

            for i in range(len(lis_target_ind)):

                new_end = self.make_end_h(lis_w[lis_target_ind[i]], lis_t[lis_target_ind[i]], off_word[i])
                lis_w[lis_target_ind[i]] = new_end
            
            
#             jam.jamo.append(w_last)
            
            res = ' '.join(lis_w)
            jam = Jamodealer(res)
            
            
            return jam.make_one()

        return input


    
        
    def processText(self,stc):
        result = stc
        
        flag = 0
        if result[-1]=='\n':
            result = result.replace('\n','')
        
        num = 0
        while 1:
            if result[-1-num]!=' ':
                break
            else:
                num = num+1
                
        
                
        if num==0:
            rere = result
        else:
            rere = result[:-num]
            
        
        r_pun = ''
        r_word = rere
        while True:
            if r_word[-1] in SV_LIST:
                r_pun = r_pun+r_word[-1]
                r_word = r_word[:-1]
            else:
                break
        
        
        num_space = 0
        for i in r_word:
            if i==' ':
                num_space = num_space+1
            else:
                break
            
        if num_space!=0:
            r_word = r_word[num_space:]

#         cc = ch.processText(line)
    
        plus = ''
        for s in range(num_space):
            plus = plus+' '
    
        if r_word[-1] =='?' or r_word[-1] =='.' or r_word[-1] =='!' or r_word[-1] =='\"':
            r_word = r_word
        else:
            r_word = r_word+'.'
            flag = 1
            
        try:
            res = self.to_high(r_word)
        except:
            res = r_word
            
        r_word = plus+r_word
        res = plus+res
        
        
        
#         if r_word[-1] =='?' or r_word[-1] =='.' or r_word[-1] =='!' or r_word[-1] =='\"':
#             r_word = r_word
#         else:
#             r_word = r_word+'.'
#             flag = 1
        
#         res = self.to_high(r_word)
        
        if flag ==1:
            res = res[:-1]
        
        #spacing = Spacing()
        #res = spacing(res)
        return res+r_pun[::-1]

In [29]:
ch = Changer()
ch.to_high('올라.')

'올라.'

In [23]:
f0 = open('./아카이브/aihub.kr.shuf', 'r', encoding = 'utf-8')
f1 = open('./aihub.mecab_speed_test_01.shuf', 'w',encoding = 'utf-8')



flag = 0

num = 0
number = 0
ch = Changer()

while True:
    flag = 0
    line = f0.readline()
    if not line: break
    if line[-1]=='\n':
        line = line.replace('\n','')
    
    num_space = 0
    for i in line:
        if i==' ':
            num_space = num_space+1
        else:
            break
            
    if num_space!=0:
        line = line[num_space:]
    
#     try:
    cc = ch.processText(line)
#     except:
#         cc = line
    
    plus = ''
    for s in range(num_space):
        plus = plus+' '
    
    line = plus+line
    cc = plus+cc
    f1.write(cc)
    f1.write('\n')
    num = num+1
    if cc!=line:
        number = number+1
    print(num)
#     else:
#         break
#    print(number)

f0.close()
f1.close()

FileNotFoundError: [Errno 2] No such file or directory: './아카이브/aihub.kr.shuf'

In [16]:
kr = open('./aihub.mecab_speed_test_01.shuf', "r" , encoding = "utf-8")
origin = open('./아카이브/aihub.kr.shuf', "r" , encoding = "utf-8")


부호 = ['.', '?', '!', ')', '"', '…', '”', '．', ':', '~', ']', '┃', ',', '」']

요 = 0
다 = 0
까 = 0
나머지 = 0
count = 0

result=kr.readline()
ori = origin.readline()


    
while True:

    
    
    result = result.replace('\n','')
    ori = ori.replace('\n','')
    if not result: break
    
    if 'ㅂ니다' in result:
        count = count+1
        print(result)
        print(ori)
        print(count)
        
        
    result=kr.readline()
    ori = origin.readline()

kr.close()

“가난한 사람이 경작한 밭에서는 많은 소출이 날 수도 있으나 불의가 판을 치면 그에게 돌아갈 몫이 없.”(13:23ㅂ니다, 새번역) 정규직을 꿈꾸며 열심히 일하던 비정규직 청년이 불의한 구조로 모든 것을 잃게 된 것입니다.
“가난한 사람이 경작한 밭에서는 많은 소출이 날 수도 있으나 불의가 판을 치면 그에게 돌아갈 몫이 없다.”(13:23, 새번역) 정규직을 꿈꾸며 열심히 일하던 비정규직 청년이 불의한 구조로 모든 것을 잃게 된 것이다.
1
스타벅스 국내 매장이 1년간 사용하는 플라스틱 빨대(길이 21㎝)만 약 1억8000만개로 이를 연결하면 지구를 한 바퀴 감쌀 정도의 길이(3만8700㎞ㅂ니다).
스타벅스 국내 매장이 1년간 사용하는 플라스틱 빨대(길이 21㎝)만 약 1억8000만개로 이를 연결하면 지구를 한 바퀴 감쌀 정도의 길이(3만8700㎞)다.
2
그중 대표적인 작품이 2017년 고야 단편 애니메이션 상을 수상한 ‘짧은 하루:어라이크(ALIKEㅂ니다)’.
그중 대표적인 작품이 2017년 고야 단편 애니메이션 상을 수상한 ‘짧은 하루:어라이크(ALIKE)’다.
3
내 생각에 최고의 티브이 프로그램은 리얼리티 2ㅂ니다.
내 생각에 최고의 티브이 프로그램은 리얼리티 2다.
4
이 조례에 따라 구세를 감면받고자 하는 자는 「지방세특례제한법 시행규칙」 별지 제1호 서식에 따른 지방세 감면신청서 및 그 사실을 증명할 수 있는 서류를 갖추어 구청장에게 신청하여야 합니다. 다만, 구청장이 감면대상임을 알 수 있는 때에는 신청이 없는 경우라도 직권으로 감면할 수 있.②ㅂ니다 구청장이 제1항에 따른 신청을 받은 때에는 감면여부를 조사·결정하고 그 내용을 「지방세특례제한법 시행규칙」 별지 제2호 서식에 따라 신청인에게 통지하여야 합니다.
이 조례에 따라 구세를 감면받고자 하는 자는 「지방세특례제한법 시행규칙」 별지 제1호 서식에 따른 지방세 감면신청서 및 그 사실을 증명할 수 있는 서류를 갖추어 구청장에게 신청하여야 한다. 다만, 구청장이 감면대상임을 알 수 있는 때에는

In [13]:
def prepro_ch02(input, lis_ef, tag_last, lis_w_last, lis_w_last_not):
    
    lis_res_word = []
    
    lis_input = input.split()
    lis_word = []
    lis_tag = []
    lis_last_word = []
    lis_ind = []
    
    lis_result = []
    
    for i in range(len(lis_input)):
        ele_w = []
        ele_t = []
        
        an = mec.pos(lis_input[i])
        for j in range(len(an)):
            ele_w.append(an[j][0])
            ele_t.append(an[j][1])
        elem_w = ' '.join(ele_w)
        elem_t = '/'.join(ele_t)
        
        jam_pre = Jamodealer(elem_w)
        lis_word.append(''.join(jam_pre.jamo))
        lis_tag.append(elem_t)
    
    for i in range(len(lis_tag)):
        
        if 'EF/SF' in lis_tag[i] or 'EF/SV' in lis_tag[i]:
            elemen_t = lis_tag[i].split('/')
            elemen_w = lis_word[i].split(' ')
            flag = 0
            
            
            for j in range(len(elemen_t)):

                flag_end = detect_h(elemen_w[j], lis_w_last_not, lis_w_last)

                
                if flag_end == 1 and 'EF' in elemen_t[j]:
                    
                    for jam in lis_w_last_not:
                        
                        if len(elemen_w[j])>=len(jam):
                            
                            if elemen_w[j][-len(jam):]==jam:
                                
                                
                                lis_ind.append(i)
                                lis_last_word.append(jam)
                                elemen_w[j] = elemen_w[j][:-len(jam)]
                                
                                break
                                
            res_w = ''.join(elemen_w)
            lis_result.append(res_w)
                                
        else:
            
            rere = lis_word[i].split(' ')
            
            resres = ''.join(rere)
            
            lis_result.append(resres)
            
            
    return lis_result, lis_tag, lis_ind, lis_last_word
     

In [15]:
print(prepro_ch02('이번주 금요일에 그곳에 갈 것입니다.', lis_beta_ef, lis_tag_last, lis_end_2low, lis_end))

(['ㅇㅣㅂㅓㄴㅈㅜ', 'ㄱㅡㅁㅇㅛㅇㅣㄹㅇㅔ', 'ㄱㅡㄱㅗㅅㅇㅔ', 'ㄱㅏㄹ', 'ㄱㅓㅅㅇㅣ.'], ['NNG/NNG', 'NNG/JKB', 'NP/JKB', 'VV+ETM', 'NNB/VCP+EF/SF'], [4], ['ㅂㄴㅣㄷㅏ'])
